In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [4]:
mnist_train = dset.MNIST("./",train=True,transform = transforms.ToTensor(),target_transform=None,download=False)
mnist_test = dset.MNIST("./",train=False,transform = transforms.ToTensor(),target_transform=None,download=False)

In [5]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,
                                          drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,
                                          drop_last=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(nn.Conv2d(1,16,5),# Conv2d in  out kernel stride padding
                                  nn.ReLU(),
                                  nn.Conv2d(16,32,5),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2,2), # MaxPool2d kernelsize,stride
                                  nn.Conv2d(32,64,5),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2,2))
        self.fc_layer = nn.Sequential(nn.Linear(64*3*3,100),
                                     nn.ReLU(),
                                     nn.Linear(100,10))
    
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out
        

In [10]:
model = CNN()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [17]:
loss_arr=[]
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image
        y = label
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        if j == 30 :
            break
        if j % 100 == 0 :
            print(loss)
            loss_arr.append(loss.detach().numpy())

tensor(0.7180, grad_fn=<NllLossBackward>)
tensor(0.4699, grad_fn=<NllLossBackward>)
tensor(0.4067, grad_fn=<NllLossBackward>)
tensor(0.3238, grad_fn=<NllLossBackward>)
tensor(0.4260, grad_fn=<NllLossBackward>)
tensor(0.2620, grad_fn=<NllLossBackward>)
tensor(0.3056, grad_fn=<NllLossBackward>)
tensor(0.2137, grad_fn=<NllLossBackward>)
tensor(0.2049, grad_fn=<NllLossBackward>)
tensor(0.1990, grad_fn=<NllLossBackward>)


In [48]:
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image
        y = label
        output = model.forward(x)
        _,output_index = torch.max(output,1) #torch.max (output, dim) output == output_val , output_index(class)
        
        total += label.size(0) # == label.size()[0]
        correct += (output_index == y).sum().float()
    
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 95.00200653076172


In [46]:
sample[0][0].size(2)

28

In [22]:
sample[0][0].size()

torch.Size([1, 28, 28])